### 1) Valid claims for current track
* Current plan: SIAC tested as a **Fourier/B-spline convolution filter** (a kernel smoother with polynomial reproduction), not as a DG superconvergence extractor.

What can be claimed within this plan/track:
* SIAC kernels are **compactly supported, smooth B-spline filters** with **polynomial reproduction** up to degree $r$.
* This acts as a controlled **low-pass / denoising operator** that can reduce oscillatory artifacts and noise, at the cost of some bias (edge smoothing).

What will not be true (no actual DG solution):
* DG superconvergence recovery $(2p+1)$ because we're not filtering a DG solution from a DG method

### 2) Where to apply SIAC: data space vs image space
In tomography, you can apply smoothing at three different points. All three are worth testing.



##### A) SIAC in sinogram space (pre-reconstruction)

**Pipeline**
\begin{equation*}
    g^{\text{obs}} \overset{\text{SIAC}_s}{\longrightarrow} \tilde g \overset{\text{FBP}}{\longrightarrow} \tilde u
\end{equation*}

Pros:
* Can supress noise before inversion (which amplifies high frequencies)
* Can be more effective for poisson noise
Cons: 
* Smoothing here is anisotropic: along detector direction vs angle direction differs
* Over smoothing can cause strong blurring effects in reconstructed image

Start with 1D SIAC smoothing along detector bins for each angle

##### B) SIAC in image space (post-processing)
**Pipeline**
\begin{equation*}
    g^{\text{obs}} \overset{\text{FBP}}{\longrightarrow} u^{\text{FBP}} \overset{\text{SIAC}_x}{\longrightarrow} u^*
\end{equation*}

Pros: 
* Easy, directly comparable to standard image denoisers.
* Analyzing frequency response can be done cleanly.

Cons: 
* FBP already amplifies certain frequencies. Filtering after the fact may be less effective.

##### C) Both (pre + post)
**Pipeline**
\begin{equation*}
    g^{\text{obs}} \overset{\text{SIAC}_s}{\longrightarrow} \tilde g \overset{\text{FBP}}{\longrightarrow} \tilde u  \overset{\text{SIAC}_x}{\longrightarrow} u^*
\end{equation*}

Pros:
* Could give best noise suppression.
Cons:
* High risk of bias / blurring.

### 3) About adding and subtracting SIAC
Let $\mathcal{S}$ be the SIAC filter in image space acting on both "$x$" and "$y$" directions.

Define the sharpening / high-boost operator
$$
    u^{-} = u - \big(\mathcal{S}(u) - u \big) = 2u - \mathcal{S}(u)
$$
* Sharpens edges
* Also amplifies noise

Or we can try the family associated with these types of filtering:
$$
    u_\alpha = u + \alpha\big(\mathcal{S}(u) - u \big)
$$

* $\alpha = 1$: full SIAC
* $\alpha = 0$: No SIAC
* $\alpha \lt 0$: sharpening
* $0 \lt\alpha \lt 1$: partial smoothing (could be best)

### 4) Extra possible test: SIAC regularizer

We try to solve
$$
    \min_u \tfrac{1}{2}\|Au-g\|^2 + \tfrac{\lambda}{2}\|\mathcal{S}(u)-u\|^2
$$

Using the square norm on both places yields:
* Smooth objective,
* linear normal equations,
* No subgradient methods needed.

If we let $B := (S-I)$, then we have the objective
$$
    J(u) = \tfrac{1}{2}\|Au-g\|^2 + \tfrac{\lambda}{2} \|Bu\|^2
$$
And the Gradient becomes
$$
    \nabla J(u) = A^* (Au-g) + \lambda B^*Bu
$$
So we need $A^*$ and $B^*$. 

Since SIAC is a convolution:
$$
    \mathcal{S}(u) = k \ast u
$$
In discrete periodic setting, using FFT:
$$
    \widehat{Su}(\xi) = \hat k (\xi) \hat u(\xi)
$$
We also have the adjoint property in $\ell^2$:
$$
    S^* \text{corresponds to multiplication by } \overline{\hat k(\xi)}
$$
And if $\mathcal{S}$ is self-adjoint, then
$$
    B^*B = (\mathcal{S}-I)^2
$$